In [ ]:
!pip install openai langchain_community langchain_core langchain_openai pypdf

In [ ]:
!pip install langchain

In [ ]:
import langchain

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "dummy_key"

In [ ]:
MODEL = "llama2"

# from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
# from langchain_openai.embeddings import OpenAIEmbeddings

model = ChatOllama(model=MODEL, temperature=0.3)
embeddings = OllamaEmbeddings(model=MODEL,)

model.invoke("Tell me a joke")

In [ ]:
model.invoke("tell me another one")

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
# chain.invoke("What can you tell me about Nigeria's Inflation")

In [ ]:
from langchain.prompts import PromptTemplate

template = """
You are a helpful assistant, who has been orderined by the power customer complaint about food, brands and Kitchen.
You are to help Answer questions based on the Complaint Details that will be provided in the context. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

In [ ]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Abubakar but I like to go by HIM", "question": "What's your name'?"})

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

csv_loader = CSVLoader(file_path="../reviews.csv", source_column="text", csv_args={
        "delimiter": ",",
        "fieldnames": ["class", "text"],
    },)
csv_df = csv_loader.load()
csv_df

In [ ]:
len(csv_df)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../lmv_training.pdf")
pages = loader.load_and_split()
pages

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch
def create_doc_array_in_memory(documents, embeddings_type=embeddings):
    vectorstore = DocArrayInMemorySearch.from_documents(documents, embedding=embeddings_type)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
    return retriever

In [ ]:
retriever = create_doc_array_in_memory(csv_df)

In [ ]:
retriever_lmv = create_doc_array_in_memory(pages)

In [ ]:
retriever.invoke("a unique film that played completely on its own merit")

In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("Training Curriculum Information")

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [ ]:
questions = ["List 20 movies with negative class. Only the movie name and the director",
             "How many movies has a positive class?",
             "how many movies were released between the year 1990 and 2010?"]

In [ ]:
for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

## Reviews RAG

In [ ]:
reviews_retriever = create_doc_array_in_memory(csv_df)

In [ ]:
# test retriever
reviews_retriever.invoke("sympathetic characters")

In [ ]:
from operator import itemgetter

review_chain = (
    {
        "context": itemgetter("question") | reviews_retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)